# Combining CSV(s)

In [ ]:
import os
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup
!pip install langdetect
from langdetect import detect

file_path = "/content/drive/MyDrive/USYD/SCDL1991/SCDL 1991 Group 10 Folder/Comments Data/All Platforms Comments/facebook_csv/Reels/"

file_list = [f for f in os.listdir(file_path) if not f.startswith('.')]

# dealing with inconsistent file fields
file_list_9 = []
file_list_11 = []

data = []

for i in range(len(file_list)):
  filename = file_list[i]
  if filename.endswith(".csv"):
      fp = os.path.join(file_path, filename)

      with open(fp, 'r', encoding='ISO-8859-1') as f:
        header_line = f.readline().strip().split(',')
        num_columns = len(header_line)

        if num_columns == 9:
          file_list_9.append(filename)
        elif num_columns == 11:
          file_list_11.append(filename)

      df = pd.read_csv(fp, encoding='ISO-8859-1')

      # num_likes = df['Likes'].sum()
      num_rows = len(df)

      data.append({'Post': filename, '# of Comments': num_rows})

print(pd.DataFrame(data))
print(file_list_9)
print(file_list_11)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993227 sha256=9e518f52fcb6a838f8bb945de8ff90fb31c36bde1e7856c5572b209d0f87b2bd
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect
                   Post  # of Comments
0    facebookreels1.csv             52
1    facebookreels3.csv             55
2    facebookreels4.csv             55
3    facebookreels9.csv             22
4   facebookreels11.csv             26
5   facebookreels13.csv             17
6   facebookreels14.csv             13
7   facebookreels17.csv             15
8   facebookreels18.csv             44
9   facebookreels16.csv            106
10  facebookreels20.csv             57
11  facebookreels15.csv             33
['facebookreels11.csv', 'facebookreels13.csv', 'facebookreels14.csv', 'fac

In [ ]:
# template: extract title from url
import requests
from bs4 import BeautifulSoup

url = 'https://m.facebook.com/share/BNk3ozf6QQr4p4qN/?mibextid=Cx5MWH'
response = requests.get(url)

soup = BeautifulSoup(response.content, 'html.parser')
title = soup.title.string

print(title)

Log into Facebook | Facebook


In [ ]:
def is_english(comment):
    try:
        return detect(comment) == 'en'  # 'en' represents English language code
    except:
        return False

## Facebook

In [ ]:
def read_csv_9(file):
    df_tmp = pd.read_csv(file_path + file, encoding="ISO-8859-1")
    # Extract URL from the second row and create a new column
    url = df_tmp.iloc[0, 1]

    df = pd.read_csv(file_path + file, header=6, encoding="ISO-8859-1")
    df['post_url'] = url

    # remove duplicates and empty
    df.drop_duplicates(subset=['Comment'], inplace=True)
    df = df[df['Comment'].notna()]

    # Filter out non-English comments
    df = df[df['Comment'].apply(is_english)]

    # sort by most recent to latest
    df['Date'] = pd.to_datetime(df['Date'], format='mixed')
    df.sort_values(by='Date', ascending=False, inplace=True)
    df = df.head(100)

    return df

df_concat = pd.concat([read_csv_9(f) for f in file_list if os.path.isfile(file_path + f)], ignore_index=True)

field_dropped = ["unnamed", "isHearted", "isPinned", "(view source)", "Profile ID", "Comment ID", "Comment URL", "ï»¿", "Stars"]
for n in field_dropped:
  df_concat.drop(df_concat.columns[df_concat.columns.str.contains(n,case = False)],axis = 1, inplace = True)

df_concat['Likes'].fillna(0, inplace=True)

df_concat.rename(columns={'Name (click to view profile)': 'Name'}, inplace=True)
df_concat.rename(columns={'Likes': 'comment_likes'}, inplace=True)
df_concat.rename(columns={'Comment URL': 'post_url'}, inplace=True)
df_concat.rename(columns={'Name': 'comment_author'}, inplace=True)
df_concat.rename(columns={'Comment': 'comment_text'}, inplace=True)
df_concat.rename(columns={'Date': 'comment_time'}, inplace=True)

df_concat

<ipython-input-5-38e242134907>:27: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_concat.drop(df_concat.columns[df_concat.columns.str.contains(n,case = False)],axis = 1, inplace = True)


,comment_author,comment_time,comment_likes,comment_text,post_url
0,Mike Ghouse,2020-02-04 03:27:19,1.0,"Love this statement, âQuran is a fixed text ...",https://m.facebook.com/share/BNk3ozf6QQr4p4qN/...
1,Robert Crane,2020-02-04 02:25:22,1.0,Brilliant interpretation. QurÃ¡n 51:47 on an ...,https://m.facebook.com/share/BNk3ozf6QQr4p4qN/...
2,Saddiq India,2020-01-04 16:20:22,1.0,Yes Heaven is raised which no eyes had ever se...,https://m.facebook.com/share/BNk3ozf6QQr4p4qN/...
3,Wasim Inamdar,2020-01-04 03:10:46,1.0,Thanks for the great article....!ð,https://m.facebook.com/share/BNk3ozf6QQr4p4qN/...
4,Steven W. De Bernardi,2020-01-04 02:06:07,1.0,Your translation choices in the 21st century a...,https://m.facebook.com/share/BNk3ozf6QQr4p4qN/...
...,...,...,...,...,...
406,Indi Mallinder-Capamagian,2021-09-18 05:26:59,0.0,Tazman-Jack Nesbit,https://m.facebook.com/share/p/M7jmHX2EsBJvyyAP/?
407,Brian Otieno,2021-09-18 03:33:19,0.0,I just don't get how there equipment can get i...,https://m.facebook.com/share/p/M7jmHX2EsBJvyyAP/?
408,???,2021-09-18 03:14:33,0.0,"He is lack, so he is not enough.",https://m.facebook.com/share/p/M7jmHX2EsBJvyyAP/?
409,Hong Cong Phan,2021-09-18 02:59:47,2.0,You say Milky way's shredded provide clue abou...,https://m.facebook.com/share/p/M7jmHX2EsBJvyyAP/?


In [ ]:
new_df = df_concat[['post_url']]
new_df.drop_duplicates(subset=['post_url'], inplace=True)

post_data = {
    'https://m.facebook.com/share/6b922Mszv8RU9VuC/?mibextid=Cx5MWH': 'Shedding Light on The Dark Matter',
    'https://m.facebook.com/share/BNk3ozf6QQr4p4qN/?mibextid=Cx5MWH': 'Safi Kaskas - Dark Energy, Dark Matter',
    'https://m.facebook.com/share/p/CtL5vD9ph3w95wkt/?mibextid=UIerB5': '''Astrophysicist's research could provide a hint in the search for dark matter''',
    'https://m.facebook.com/share/v/pZ4ug4wCXryDcSa8/?mibextid=Cx5MWH': '''Mystery of Galaxy's Missing Dark Matter Deepens''',
    'https://m.facebook.com/share/4iGhQYkgLyKWRFeZ/?mibextid=Cx5MWH': 'Fraternali’s team unveils galaxies defying traditional dark matter models, challenging long-held astronomical beliefs',
    'https://m.facebook.com/share/p/iRoQ9vEGh9vYEKMj/?mibextid=UIerB5': 'Recent research conducted by scientists from the University of Granada can contribute to determine the nature of dark matter, one of the most important mysteries in physics',
    'https://m.facebook.com/share/hmwoAXeRcMqBVibM/?mibextid=Cx5MWH': 'An international team of researchers has created the largest and most detailed map of the distribution of so-called dark matter in the Universe',
    'https://m.facebook.com/share/v/NQ6GUWuNpAXWh6mf/?mibextid=Cx5MWH': 'Hubble Makes Unexpected Dark Matter Discovery',
    'https://m.facebook.com/share/f6tmYxGUVe3GoZL5/?mibextid=Cx5MWH': 'Super fun discussion about the science of Dark Matter. Also: some cool, new images from the show...',
    'https://m.facebook.com/share/v/bkw5F1S9qvwR8LEb/?': 'How dark matter connects galaxies',
    'https://m.facebook.com/share/p/M7jmHX2EsBJvyyAP/?': 'Scientists have mapped Sagittarius—a neighboring dwarf galaxy—in exquisite detail, and they’ve used that map to provide a long-sought picture of the mysterious dark matter halo in which our Milky Way resides',
    'https://m.facebook.com/share/v/VFiA2n91KseUrKQs/?mibextid=Cx5MWH': 'Astronomers discovered a galaxy that has no dark matter',
    'https://m.facebook.com/share/po9qxEvGjjYHXhks/?mibextid=Cx5MWH': '"Dark matter and dark energy are vivid reminders that science is an ongoing flirtation with the unknown."',
    'https://m.facebook.com/share/AvT5UTiyzkybFbDL/?mibextid=Cx5MWH': '🗞️Controversial new theory of gravity rules out need for dark matter 🗞️',
    'https://m.facebook.com/share/v/R2hJ64g7QWbBE53m/?mibextid=Cx5MWH': 'The latest dark matter science results report on the warped light of distant galaxies, which ironically leave us more in the dark than ever',
    'https://m.facebook.com/share/Ujf7zPWXSZTmHgm5/?mibextid=Cx5MWH': '''Lone black holes are massive but dark — a tempting explanation for dark matter. Here's why astronomers don't believe these objects are the culprit''',
    'https://m.facebook.com/share/SpTFFmTThGX9xS3R/?mibextid=Cx5MWH': 'While astronomers and physicists widely accept the existence of dark matter today, many recent creationists oppose the existence of dark matter',
    'https://m.facebook.com/share/p/QHPdRx7srhfiX3d1/?mibextid=UIerB5': 'The Mysterious Dark Matter',
    'https://m.facebook.com/share/qUmZmvy71kf21osf/?mibextid=Cx5MWH': 'Confused about dark matter and dark energy? This is the key difference between the two'
}

def extract_title_from_url(url):
  title = post_data.get(url)
  return title

new_df['post_title'] = new_df['post_url'].apply(extract_title_from_url)
new_df

<ipython-input-9-77656186a1ca>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.drop_duplicates(subset=['post_url'], inplace=True)
<ipython-input-9-77656186a1ca>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['post_title'] = new_df['post_url'].apply(extract_title_from_url)


,post_url,post_title
0,https://m.facebook.com/share/BNk3ozf6QQr4p4qN/...,"Safi Kaskas - Dark Energy, Dark Matter"
5,https://m.facebook.com/share/p/CtL5vD9ph3w95wk...,Astrophysicist's research could provide a hint...
9,https://m.facebook.com/share/4iGhQYkgLyKWRFeZ/...,Fraternali’s team unveils galaxies defying tra...
22,https://m.facebook.com/share/p/iRoQ9vEGh9vYEKM...,Recent research conducted by scientists from t...
24,https://m.facebook.com/share/hmwoAXeRcMqBVibM/...,An international team of researchers has creat...
99,https://m.facebook.com/share/v/NQ6GUWuNpAXWh6m...,Hubble Makes Unexpected Dark Matter Discovery
173,https://m.facebook.com/share/f6tmYxGUVe3GoZL5/...,Super fun discussion about the science of Dark...
188,https://m.facebook.com/share/v/VFiA2n91KseUrKQ...,Astronomers discovered a galaxy that has no da...
205,https://m.facebook.com/share/po9qxEvGjjYHXhks/...,"""Dark matter and dark energy are vivid reminde..."
219,https://m.facebook.com/share/v/R2hJ64g7QWbBE53...,The latest dark matter science results report ...


In [ ]:
new_df.to_csv('/content/drive/MyDrive/USYD/SCDL1991/SCDL 1991 Group 10 Folder/Comments Data/dict_facebook_post.csv', index=False)

In [ ]:
df_concat.to_csv('/content/drive/MyDrive/USYD/SCDL1991/SCDL 1991 Group 10 Folder/Comments Data/Facebook_post.csv', index=False)

## Facebook reels

In [ ]:
def read_csv_9(file):
    df_tmp = pd.read_csv(file_path + file, encoding="ISO-8859-1")
    # Extract URL from the second row and create a new column
    url = df_tmp.iloc[0, 1]

    df = pd.read_csv(file_path + file, header=6, encoding="ISO-8859-1")
    df['post_url'] = url

    # remove duplicates and empty
    df.drop_duplicates(subset=['Comment'], inplace=True)
    df = df[df['Comment'].notna()]

    # Filter out non-English comments
    df = df[df['Comment'].apply(is_english)]

    # sort by most recent to latest
    df['Date'] = pd.to_datetime(df['Date'], format='mixed')
    df.sort_values(by='Date', ascending=False, inplace=True)
    df = df.head(100)

    return df

df_concat = pd.concat([read_csv_9(f) for f in file_list if os.path.isfile(file_path + f)], ignore_index=True)

field_dropped = ["unnamed", "isHearted", "isPinned", "(view source)", "Profile ID", "Comment ID", "Comment URL", "ï»¿", "Stars"]
for n in field_dropped:
  df_concat.drop(df_concat.columns[df_concat.columns.str.contains(n,case = False)],axis = 1, inplace = True)

df_concat['Likes'].fillna(0, inplace=True)

df_concat.rename(columns={'Name (click to view profile)': 'Name'}, inplace=True)
df_concat.rename(columns={'Likes': 'comment_likes'}, inplace=True)
df_concat.rename(columns={'Comment URL': 'post_url'}, inplace=True)
df_concat.rename(columns={'Name': 'comment_author'}, inplace=True)
df_concat.rename(columns={'Comment': 'comment_text'}, inplace=True)
df_concat.rename(columns={'Date': 'comment_time'}, inplace=True)

df_concat

In [ ]:
new_df = df_concat[['post_url']]
new_df.drop_duplicates(subset=['post_url'], inplace=True)

def extract_title_from_url(url):
  response = requests.get(url)
  soup = BeautifulSoup(response.content, 'html.parser')
  title = soup.title.string
  return title

new_df['post_title'] = new_df['post_url'].apply(extract_title_from_url)
new_df

In [ ]:
new_df.to_csv('/content/drive/MyDrive/USYD/SCDL1991/SCDL 1991 Group 10 Folder/Comments Data/dict_facebook_reel.csv', index=False)

In [ ]:
df_concat.to_csv('/content/drive/MyDrive/USYD/SCDL1991/SCDL 1991 Group 10 Folder/Comments Data/Facebook_reel.csv', index=False)

## Youtube

In [ ]:
# Youtube (need to deal with files with 11 columns (inconcsistent data fields))
yt1_source = 'https://www.youtube.com/watch?v=HneiEA1B8ks'
yt3_source = 'https://www.youtube.com/watch?v=cdqqn9o7XS4'

def read_csv_9(file):
    df_tmp = pd.read_csv(file_path + file, encoding="ISO-8859-1")
    # Extract URL from the second row and create a new column
    url = df_tmp.iloc[0, 1]

    df = pd.read_csv(file_path + file, header=6, encoding="ISO-8859-1")
    df['post_url'] = url

    # remove duplicates and empty
    df.drop_duplicates(subset=['Comment'], inplace=True)
    df = df[df['Comment'].notna()]

    # Filter out non-English comments
    df = df[df['Comment'].apply(is_english)]

    # sort by most recent to latest
    df['Date'] = pd.to_datetime(df['Date'], format='mixed')
    df.sort_values(by='Date', ascending=False, inplace=True)
    df = df.head(100)

    return df

def read_csv_11(file):
    df = pd.read_csv(file_path + file, encoding="ISO-8859-1")
    if file == 'Youtube Video 1.csv':
      url = yt1_source
    elif file == 'Youtube Video 3_csv.csv':
      url = yt3_source

    df['post_url'] = url

    # sort by most recent to latest
    df['Date'] = pd.to_datetime(df['Date'], format='mixed')
    df.sort_values(by='Date', ascending=False, inplace=True)
    df = df.head(100)

    return df

df_concat_11 = pd.concat([read_csv_11(f) for f in file_list_11 if os.path.isfile(file_path + f)], ignore_index=True)
df_concat_9 = pd.concat([read_csv_9(f) for f in file_list_9 if os.path.isfile(file_path + f)], ignore_index=True)

field_dropped = ["unnamed", "isHearted", "isPinned", "(view source)", "Profile ID", "Comment ID", "Comment URL", "ï»¿"]
for n in field_dropped:
  df_concat_9.drop(df_concat_9.columns[df_concat_9.columns.str.contains(n,case = False)],axis = 1, inplace = True)
  df_concat_11.drop(df_concat_11.columns[df_concat_11.columns.str.contains(n,case = False)],axis = 1, inplace = True)

df_concat = pd.concat([df_concat_9, df_concat_11], ignore_index=True)
df_concat.rename(columns={'Name (click to view profile)': 'Name'}, inplace=True)

df_concat.rename(columns={'Likes': 'comment_likes'}, inplace=True)
df_concat.rename(columns={'Comment URL': 'post_url'}, inplace=True)
df_concat.rename(columns={'Name': 'comment_author'}, inplace=True)
df_concat.rename(columns={'Comment': 'comment_text'}, inplace=True)
df_concat.rename(columns={'Date': 'comment_time'}, inplace=True)

df_concat

<ipython-input-10-1a5ecb6a6025>:48: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_concat_9.drop(df_concat_9.columns[df_concat_9.columns.str.contains(n,case = False)],axis = 1, inplace = True)
<ipython-input-10-1a5ecb6a6025>:49: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_concat_11.drop(df_concat_11.columns[df_concat_11.columns.str.contains(n,case = False)],axis = 1, inplace = True)


,comment_author,comment_time,comment_likes,comment_text,post_url
0,@mikilynne4558,2024-11-02 16:42:04,0.0,The incomprehensible distances between galaxie...,https://www.youtube.com/watch?v=uBbxXNhZ78c&pp...
1,@ravencroft99,2024-11-01 14:01:03,0.0,Could a particle moving faster than light have...,https://www.youtube.com/watch?v=uBbxXNhZ78c&pp...
2,@francishubertovasquez2139,2024-06-01 06:58:26,0.0,Dark Matter and Dark Energy are my footstool.,https://www.youtube.com/watch?v=uBbxXNhZ78c&pp...
3,@flyfishing739,2024-05-01 00:19:43,0.0,The James Webb proves this is not true as huge...,https://www.youtube.com/watch?v=uBbxXNhZ78c&pp...
4,@mt-qc2qh,2024-04-17 09:25:50,1.0,Maybe they will finally hypothesize that the c...,https://www.youtube.com/watch?v=uBbxXNhZ78c&pp...
...,...,...,...,...,...
1593,@KeepingWatch95,2024-03-19 02:55:00,0.0,Not any dark matter?\r\n\r\nGoing against the ...,https://www.youtube.com/watch?v=cdqqn9o7XS4
1594,@brucewilson8097,2024-03-19 01:48:00,1.0,"Refreshing, I have often thought, how long do ...",https://www.youtube.com/watch?v=cdqqn9o7XS4
1595,@TexasTimeLord,2024-03-19 00:11:00,0.0,Scientists should have realized they got the a...,https://www.youtube.com/watch?v=cdqqn9o7XS4
1596,@garylaw5874,2024-03-19 00:10:00,0.0,The under estimation of the mass and amount of...,https://www.youtube.com/watch?v=cdqqn9o7XS4


In [ ]:
new_df = df_concat[['post_url']]
new_df.drop_duplicates(subset=['post_url'], inplace=True)

def extract_title_from_url(url):
  response = requests.get(url)
  soup = BeautifulSoup(response.content, 'html.parser')
  title = soup.title.string
  return title

new_df['post_title'] = new_df['post_url'].apply(extract_title_from_url)
new_df

In [ ]:
new_df.to_csv('/content/drive/MyDrive/USYD/SCDL1991/SCDL 1991 Group 10 Folder/Comments Data/dict_yt_video.csv', index=False)

In [ ]:
# create a new file of all comments combined
df_concat.to_csv('/content/drive/MyDrive/USYD/SCDL1991/SCDL 1991 Group 10 Folder/Comments Data/YouTube_video.csv', index=False)

## Youtube shorts

In [ ]:
# Youtube (need to deal with files with 11 columns (inconcsistent data fields))
# 'Youtube Short 1.csv', 'Youtube Short 2.csv', 'Youtube Short 3.csv', 'Youtube Short 4.csv', 'Youtube Short 6.csv'
yt_short1_source = 'https://www.youtube.com/watch?v=Qy8CqRGDqeQ'
yt_short2_source = 'https://www.youtube.com/watch?v=PIdgqyNOl7Q'
yt_short3_source = 'https://www.youtube.com/watch?v=YSThUhFMiJY'
yt_short4_source = 'https://www.youtube.com/watch?v=-GEZ95TJ5Ao'
yt_short6_source = 'https://www.youtube.com/watch?v=aoe6BC6TZMg'


def read_csv_9(file):
    df_tmp = pd.read_csv(file_path + file, encoding="ISO-8859-1")
    # Extract URL from the second row and create a new column
    url = df_tmp.iloc[0, 1]

    df = pd.read_csv(file_path + file, header=6, encoding="ISO-8859-1")
    df['post_url'] = url

    # remove duplicates and empty
    df.drop_duplicates(subset=['Comment'], inplace=True)
    df = df[df['Comment'].notna()]

    # Filter out non-English comments
    df = df[df['Comment'].apply(is_english)]

    # sort by most recent to latest
    df['Date'] = pd.to_datetime(df['Date'], format='mixed')
    df.sort_values(by='Date', ascending=False, inplace=True)
    df = df.head(100)

    return df

def read_csv_11(file):
    df = pd.read_csv(file_path + file, encoding="ISO-8859-1")
    if file == 'Youtube Short 1.csv':
      url = yt_short1_source
    elif file == 'Youtube Short 2.csv':
      url = yt_short2_source
    elif file == 'Youtube Short 3.csv':
      url = yt_short3_source
    elif file == 'Youtube Short 4.csv':
      url = yt_short4_source
    elif file == 'Youtube Short 6.csv':
      url = yt_short6_source

    df['post_url'] = url

    # sort by most recent to latest
    df['Date'] = pd.to_datetime(df['Date'], format='mixed')
    df.sort_values(by='Date', ascending=False, inplace=True)
    df = df.head(100)

    return df

df_concat_11 = pd.concat([read_csv_11(f) for f in file_list_11 if os.path.isfile(file_path + f)], ignore_index=True)
df_concat_9 = pd.concat([read_csv_9(f) for f in file_list_9 if os.path.isfile(file_path + f)], ignore_index=True)

field_dropped = ["unnamed", "isHearted", "isPinned", "(view source)", "Profile ID", "Comment ID", "Comment URL", "ï»¿"]
for n in field_dropped:
  df_concat_9.drop(df_concat_9.columns[df_concat_9.columns.str.contains(n,case = False)],axis = 1, inplace = True)
  df_concat_11.drop(df_concat_11.columns[df_concat_11.columns.str.contains(n,case = False)],axis = 1, inplace = True)

df_concat = pd.concat([df_concat_9, df_concat_11], ignore_index=True)
df_concat.rename(columns={'Name (click to view profile)': 'Name'}, inplace=True)

df_concat.rename(columns={'Likes': 'comment_likes'}, inplace=True)
df_concat.rename(columns={'Comment URL': 'post_url'}, inplace=True)
df_concat.rename(columns={'Name': 'comment_author'}, inplace=True)
df_concat.rename(columns={'Comment': 'comment_text'}, inplace=True)
df_concat.rename(columns={'Date': 'comment_time'}, inplace=True)

df_concat

<ipython-input-7-25523d4ea65a>:59: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_concat_9.drop(df_concat_9.columns[df_concat_9.columns.str.contains(n,case = False)],axis = 1, inplace = True)
<ipython-input-7-25523d4ea65a>:60: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_concat_11.drop(df_concat_11.columns[df_concat_11.columns.str.contains(n,case = False)],axis = 1, inplace = True)


,comment_author,comment_time,comment_likes,comment_text,post_url
0,@billlancaste6062,2024-05-03 15:20:55,0.0,Scientists close their minds to spirituality. ...,https://youtube.com/shorts/FbdsyybTyEA?si=quzO...
1,@sihemnour6527,2024-03-30 02:35:44,0.0,"It's remarkable how with each discovery, we re...",https://youtube.com/shorts/FbdsyybTyEA?si=quzO...
2,@SteelZxx,2024-02-03 19:01:52,0.0,I have explanation of dark matter..\n\nSo basi...,https://youtube.com/shorts/FbdsyybTyEA?si=quzO...
3,@Busyguy1119,2024-01-15 16:54:24,0.0,Another Einstein is required for that,https://youtube.com/shorts/FbdsyybTyEA?si=quzO...
4,@badlydrawnsmiffy,2023-12-03 17:14:55,0.0,"Without the foreground sound fx, i would not h...",https://youtube.com/shorts/FbdsyybTyEA?si=quzO...
...,...,...,...,...,...
1318,@charlestimothyrocha3166,2024-04-02 15:52:56,0.0,School:ð\nEducational Edit:ðâ,https://www.youtube.com/watch?v=aoe6BC6TZMg
1319,@abdulrauf7601,2024-04-02 15:40:16,0.0,the cameramanðð¿,https://www.youtube.com/watch?v=aoe6BC6TZMg
1320,@guywithabunchofthoughts9,2024-04-02 15:26:31,0.0,Like the blue and red cts gojo uses,https://www.youtube.com/watch?v=aoe6BC6TZMg
1321,@user-sw9ty3kt1n,2024-04-02 14:29:54,0.0,TRUE DARKNESS not some dog shit darkness found...,https://www.youtube.com/watch?v=aoe6BC6TZMg


In [ ]:
new_df = df_concat[['post_url']]
new_df.drop_duplicates(subset=['post_url'], inplace=True)

def extract_title_from_url(url):
  response = requests.get(url)
  soup = BeautifulSoup(response.content, 'html.parser')
  title = soup.title.string
  return title

new_df['post_title'] = new_df['post_url'].apply(extract_title_from_url)
new_df

In [ ]:
new_df.to_csv('/content/drive/MyDrive/USYD/SCDL1991/SCDL 1991 Group 10 Folder/Comments Data/dict_yt_shorts.csv', index=False)

In [ ]:
# create a new file of all comments combined
df_concat.to_csv('/content/drive/MyDrive/USYD/SCDL1991/SCDL 1991 Group 10 Folder/Comments Data/YouTube_shorts.csv', index=False)

## Instagram

In [ ]:
# Insta (base)
# post_id	post_title	comment_id	comment_author	comment_text
def replace_urls(url):
    if '/c/' in url:
        return url.split('/c/')[0]  # Take everything before '/c/'

def read_csv_insta(file):
    df = pd.read_csv(file_path + file, encoding="ISO-8859-1")

    df['Comment URL'] = df['Comment URL'].apply(replace_urls)

    # remove duplicates and empty
    df.drop_duplicates(subset=['Comment'], inplace=True)
    df = df[df['Comment'].notna()]

    # Filter out non-English comments
    df = df[df['Comment'].apply(is_english)]

    # sort by most recent to latest
    df['Date'] = pd.to_datetime(df['Date'], format='mixed')
    df.sort_values(by='Date', ascending=False, inplace=True)
    df = df.head(100)
    return df

df_concat = pd.concat([read_csv_insta(f) for f in file_list if os.path.isfile(file_path + f)], ignore_index=True)

field_dropped = ["unnamed", "Username", "Likes", "ï»¿", "Profile ID", "Profile URL", "Comment ID"]
for n in field_dropped:
  df_concat.drop(df_concat.columns[df_concat.columns.str.contains(n,case = False)],axis = 1, inplace = True)

df_concat.rename(columns={'Comment URL': 'post_url'}, inplace=True)
df_concat.rename(columns={'Name': 'comment_author'}, inplace=True)
df_concat.rename(columns={'Comment': 'comment_text'}, inplace=True)
df_concat.rename(columns={'Date': 'comment_time'}, inplace=True)

df_concat

,comment_author,comment_time,comment_text,post_url
0,zmxnc_nvb,2024-03-17 21:20:24,@technolasko 'shadow universe'âï¸ð¤,https://www.instagram.com/p/C3i2ycZsINV
1,ronniefigueroa9,2024-03-17 04:45:44,"UH, Could be. Feel it's better to focus on the...",https://www.instagram.com/p/C3i2ycZsINV
2,_bts_.fandom._,2024-03-16 09:17:01,"Hey ,please follw this new account for space c...",https://www.instagram.com/p/C3i2ycZsINV
3,therealpaughshadow,2024-03-11 17:28:02,Just throwing stuff up on the wall now just in...,https://www.instagram.com/p/C3i2ycZsINV
4,bevetsbe,2024-03-11 07:36:59,"Itâs the demiurge, the demiurge is real.",https://www.instagram.com/p/C3i2ycZsINV
...,...,...,...,...
500,mamaelf2,2020-12-11 00:34:07,Wow thatâs so cool. Is it like water is rela...,https://www.instagram.com/p/CIn1zkjjqa_
501,matthew_peer,2020-12-10 23:20:47,We donât fucking know what it is but itâs ...,https://www.instagram.com/p/CIn1zkjjqa_
502,lycranizer,2020-12-10 17:53:00,You have to come up with new theories to keep ...,https://www.instagram.com/p/CIn1zkjjqa_
503,gs_rogers,2020-12-10 15:57:36,Where did you read this? I would like to read ...,https://www.instagram.com/p/CIn1zkjjqa_


In [ ]:
new_df = df_concat[['post_url']]
new_df.drop_duplicates(subset=['post_url'], inplace=True)

def extract_title_from_url(url):
  response = requests.get(url)
  soup = BeautifulSoup(response.content, 'html.parser')
  title = soup.title.string
  return title

new_df['post_title'] = new_df['post_url'].apply(extract_title_from_url)
new_df

In [ ]:
# create a new file of all comments combined
new_df.to_csv('/content/drive/MyDrive/USYD/SCDL1991/SCDL 1991 Group 10 Folder/Comments Data/dict_insta_posts.csv', index=False)

In [ ]:
# create a new file of all comments combined
df_concat.to_csv('/content/drive/MyDrive/USYD/SCDL1991/SCDL 1991 Group 10 Folder/Comments Data/Instagram_posts.csv', index=False)

## Instagram reels

In [ ]:
# Insta (base)
# post_id	post_title	comment_id	comment_author	comment_text
def replace_urls(url):
    if '/c/' in url:
        return url.split('/c/')[0]  # Take everything before '/c/'

def read_csv_9(file):
    df_tmp = pd.read_csv(file_path + file, encoding="ISO-8859-1")
    # Extract URL from the second row and create a new column
    url = df_tmp.iloc[0, 1]

    df = pd.read_csv(file_path + file, header=6, encoding="ISO-8859-1")
    df['post_url'] = url

    # remove duplicates and empty
    df.drop_duplicates(subset=['Comment'], inplace=True)
    df = df[df['Comment'].notna()]

    # Filter out non-English comments
    df = df[df['Comment'].apply(is_english)]

    # sort by most recent to latest
    df['Date'] = pd.to_datetime(df['Date'], format='mixed')
    df.sort_values(by='Date', ascending=False, inplace=True)
    df = df.head(100)

    return df

def read_csv_insta(file):
    df = pd.read_csv(file_path + file, encoding="ISO-8859-1")

    df['Comment URL'] = df['Comment URL'].apply(replace_urls)
    df.rename(columns={'Comment URL': 'post_url'}, inplace=True)

    # remove duplicates and empty
    df.drop_duplicates(subset=['Comment'], inplace=True)
    df = df[df['Comment'].notna()]

    # Filter out non-English comments
    df = df[df['Comment'].apply(is_english)]

    # sort by most recent to latest
    df['Date'] = pd.to_datetime(df['Date'], format='mixed')
    df.sort_values(by='Date', ascending=False, inplace=True)
    df = df.head(100)
    return df

# column 11
df_concat_11 = pd.concat([read_csv_insta(f) for f in file_list_11 if os.path.isfile(file_path + f)], ignore_index=True)

# column 9
df_concat_9 = pd.concat([read_csv_9(f) for f in file_list_9 if os.path.isfile(file_path + f)], ignore_index=True)

df_concat = pd.concat([df_concat_9, df_concat_11], ignore_index=True)

field_dropped = ["unnamed", "Username", "Likes", "ï»¿", "Profile ID", "Profile URL", "Comment ID", "(view source)"]
for n in field_dropped:
  df_concat.drop(df_concat.columns[df_concat.columns.str.contains(n,case = False)],axis = 1, inplace = True)

df_concat.rename(columns={'Name': 'comment_author'}, inplace=True)
df_concat.rename(columns={'Comment': 'comment_text'}, inplace=True)
df_concat.rename(columns={'Date': 'comment_time'}, inplace=True)

df_concat

<ipython-input-10-c52dbbff8871>:58: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_concat.drop(df_concat.columns[df_concat.columns.str.contains(n,case = False)],axis = 1, inplace = True)


,comment_author,comment_time,comment_text,post_url
0,andre_gomes_sci,2023-12-11 21:01:08,Why people keep saying that Dark Matter is mat...,https://www.instagram.com/reel/CvFRyO7gA-1/?ig...
1,mercuriimors,2023-11-19 17:52:52,@higgs_boson666 wonderful! I hope your researc...,https://www.instagram.com/reel/CvFRyO7gA-1/?ig...
2,higgs_boson666,2023-11-19 17:38:23,@tristis_mercurii thanks for all that knowle...,https://www.instagram.com/reel/CvFRyO7gA-1/?ig...
3,mercuriimors,2023-11-19 17:36:11,"@higgs_boson666 it is not dum, and you are sm...",https://www.instagram.com/reel/CvFRyO7gA-1/?ig...
4,higgs_boson666,2023-11-19 17:12:36,@tristis_mercurii great sorry for my Dum que...,https://www.instagram.com/reel/CvFRyO7gA-1/?ig...
...,...,...,...,...
593,a_r_upadhyay,2023-01-08 13:58:06,He is talking about me ! I m the dark matter,https://www.instagram.com/p/CnJ0ljVJUPX
594,nehazyx,2023-01-08 13:48:33,"Toh kya karu, job chorh du...those who know, k...",https://www.instagram.com/p/CnJ0ljVJUPX
595,rishabh.mangal.bansal,2023-01-08 12:41:03,"According to Vadic Rashmi Theory, universe isn...",https://www.instagram.com/p/CnJ0ljVJUPX
596,vikrai10,2023-01-08 12:16:19,"Isnât this what explained in Hindu history, ...",https://www.instagram.com/p/CnJ0ljVJUPX


In [ ]:
new_df = df_concat[['post_url']]
new_df.drop_duplicates(subset=['post_url'], inplace=True)

def extract_title_from_url(url):
  response = requests.get(url)
  soup = BeautifulSoup(response.content, 'html.parser')
  title = soup.title.string
  return title

new_df['post_title'] = new_df['post_url'].apply(extract_title_from_url)
new_df

<ipython-input-11-082dec513364>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.drop_duplicates(subset=['post_url'], inplace=True)
<ipython-input-11-082dec513364>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['post_title'] = new_df['post_url'].apply(extract_title_from_url)


,post_url,post_title
0,https://www.instagram.com/reel/CvFRyO7gA-1/?ig...,MIT | Dark matter is composed of particles tha...
66,https://www.instagram.com/reel/C5i-ludtRUa/?ig...,AstroPhilesz | Professor Brian Cox on Dark mat...
101,https://www.instagram.com/reel/C4Ba8XUqkuO/?ig...,Spacy Astro | Everything in the universe is ma...
182,https://www.instagram.com/reel/Cvu8oXHuT8s/?ig...,Athena Brensberger | Dark matter is still such...
199,https://www.instagram.com/reel/CYGIN5rJxy2/?ig...,Josh & Matt | One of our favourite sculptures ...
218,https://www.instagram.com/reel/CtPCn-cuSn9/?ig...,ESA - European Space Agency | The Universe hol...
234,https://www.instagram.com/p/CxIiFIhPyMR,| 𝗙𝗔𝗖𝗧𝗦 & 𝗞𝗡𝗢𝗪𝗟𝗘𝗗𝗚𝗘 | | What is Dark Matter?\n...
245,https://www.instagram.com/p/C2XNTUIvKam,Factsteller | Dark matter \n\n#science #scienc...
253,https://www.instagram.com/p/C4iq8Q6xasc,Rajan Chopra | Day 132/365: What is Dark Matte...
314,https://www.instagram.com/p/C5pNWLfI4C7,Sahil Mandaogane | Dark matter is a mysterious...


In [ ]:
# create a new file of all comments combined
new_df.to_csv('/content/drive/MyDrive/USYD/SCDL1991/SCDL 1991 Group 10 Folder/Comments Data/dict_insta_reels.csv', index=False)

In [ ]:
# create a new file of all comments combined
df_concat.to_csv('/content/drive/MyDrive/USYD/SCDL1991/SCDL 1991 Group 10 Folder/Comments Data/Instagram_reels.csv', index=False)

## TikTok

In [ ]:
def replace_urls(url):
    # https://www.tiktok.com/@astrokobi/video/7238956645135453442?comment_id=7238957400236442369
     if '?comment_id' in url and '/video/' in url:
        start_index = url.find('/video/') + len('/video/')  # Get the index after '/video/'
        end_index = url.find('?comment_id')  # Get the index before '?comment_id'
        if end_index != -1:
            return url[start_index:end_index]  # Extract the substring between '/video/' and '?comment_id'

def read_csv_insta(file):
    df = pd.read_csv(file_path + file, encoding="ISO-8859-1")

    df['Comment URL'] = df['Comment URL'].apply(replace_urls)

    # remove duplicates and empty
    df.drop_duplicates(subset=['Comment'], inplace=True)
    df = df[df['Comment'].notna()]

    # Filter out non-English comments
    df = df[df['Comment'].apply(is_english)]

    # sort by most recent to latest
    df['Date'] = pd.to_datetime(df['Date'], format='mixed')
    df.sort_values(by='Date', ascending=False, inplace=True)
    df = df.head(100)
    return df

df_concat = pd.concat([read_csv_insta(f) for f in file_list if os.path.isfile(file_path + f)], ignore_index=True)

field_dropped = ["unnamed", "Username", "Name", "Likes", "ï»¿", "Author URL", "Profile ID", "Comment ID"]
for n in field_dropped:
  df_concat.drop(df_concat.columns[df_concat.columns.str.contains(n,case = False)],axis = 1, inplace = True)

df_concat.rename(columns={'Comment URL': 'post_id'}, inplace=True)
df_concat.rename(columns={'Unique ID': 'comment_author'}, inplace=True)
df_concat.rename(columns={'Comment': 'comment_text'}, inplace=True)
df_concat.rename(columns={'Date': 'comment_time'}, inplace=True)

df_concat

,comment_author,comment_time,comment_text,post_id
0,tea.moth13,2023-11-26 02:40:48,"dark matter is just somtying we don't know, we...",7238956645135453442
1,batman24821,2023-08-21 02:19:56,What if you shine a light in dark matter,7238956645135453442
2,the_cooler_1,2023-06-05 21:01:25,Black Hole Sun???,7238956645135453442
3,elevateddog8477,2023-06-05 13:35:54,Dark matter is starting to be proven not to ex...,7238956645135453442
4,disastrousatoms,2023-06-05 01:57:56,This kind of stuff is hard to think about whil...,7238956645135453442
...,...,...,...,...
1519,chocyd,2020-11-16 20:58:04,How do we know what we don't know though,6895821134210632965
1520,kai.maire,2020-11-16 20:54:36,"Surprisingly, despite the vastness of the univ...",6895821134210632965
1521,1_800_kaylee,2020-11-16 20:50:04,Iâm okay with this information,6895821134210632965
1522,sewnblue,2020-11-16 20:33:59,the slow onset of panic ð,6895821134210632965


In [ ]:
new_df = df_concat[['post_id']]
new_df.drop_duplicates(subset=['post_id'], inplace=True)

post_data = {
    '7238956645135453442': 'Did we just find a Dark Matter Star? #space #spacetok #science #astronomy #astrokobi',
    '7123621853964324138': 'Replying to @faithfultrue0 Happy birthday, Vera Rubin!! 🌌💫 Discussing dark matter with @ASTRO ALEXANDRA 🪐 ! #spacetok #physicstok #astronomytok #astronomy',
    '7306469570850737413': 'DARK MATTER 🌌 #darkmatter #matter #Galaxies #solarsystem #sun #space #universe #astronomy',
    '7187771038221028651': 'Dark Matter and Dark Energy | Neil deGrasse Tyson #astrophysics#cosmos#fyp#followers#neildegrassetyson#trending#space#education#science#study',
    '7188861172634914091': 'a fun concept for a sci-fi #science #darkmatter',
    '7165886326691073282': '96% of The Universe is MISSING! #spacetok #space #darkmatter #universe #astrokobi',
    '7209310840720297218': '90% of our galaxy is MISSING… #space #spacetok #darkmatter #darkenergy #astrokobi',
    '7327913301919272197': 'The Concept about the Dark Matter ⚫🌌 #space #fypシ゚viral #fyp #fy #universe #darkmatter',
    '7150050199778676014': 'Why we don’t know what dark matter is. #space #science #physics #darkmatter #particlephysics What is dark matter What is dark energy Where is dark matter Where is dark energy',
    '6998675917799181570': 'What if space is just cosmic soil for consciousness to sprout? #consciousness #darkmatter #metaphysics #alchemy',
    '7187964392376503554': 'W matter | ps visiting family in japan for 3 weeks so i will barely be able to post but will be back 🙏 #fyp #elements',
    '7104516399095237934': 'science is fake #funny #comedy #joke #humor #meme #science #religion #politics #darkmatter #fyp',
    '6840616903778635013': 'this one gets a little DARK #science #history #physics #darkmatter #interestingfacts #fyp',
    '7061686858081946926': 'Invisible = untouchable. #DarkMatter #Physics',
    '6928113689274977542': 'We just proved why stuff probably has mass last decade most of physics is guesses and math #fifthdimension #physics #darkmatter',
    '6988015970237435141': 'Why noble gases may be the key to finding dark matter! Ft @ryan.curtis1999 #VeritasiumContest #learnontiktok #darkmatter #snolab #noblegases',
    '7260951406864125211': '''Dark matter is a hypothetical form of matter thought to account for approximately 85% of the matter in the universe. Dark matter is called "dark" because it does not appear to interact with the electromagnetic field, which means it does not absorb, reflect, or emit electromagnetic radiation and is, therefore, difficult to detect. Various astrophysical observations – including gravitational effects which cannot be explained by currently accepted theories of gravity unless more matter is present than can be seen – imply dark matter's presence. ''',
    '7264722114643578155': 'Dark Matter stars are scary… #darkmatterstar #space #astronomy #scary #universe #stem #darkmatter',
    '6992840181493615878': 'How was Dark Matter first discovered? Ft @ryan.curtis1999 #VeritasiumContest #learnontiktok #physicsfun #darkmatter #galaxies #spacetok',
    '6895821134210632965': 'I’m not having an existential crisis, what are you talking about (Host: @yourlocallibrary) #science #space #darkmatter #learnontiktok #tiktokpartner'
}

def extract_title_from_url(url):
  title = post_data.get(url)
  return title

new_df['post_title'] = new_df['post_id'].apply(extract_title_from_url)
new_df

In [ ]:
new_df.to_csv('/content/drive/MyDrive/USYD/SCDL1991/SCDL 1991 Group 10 Folder/Comments Data/dict_tiktok.csv', index=False)

In [ ]:
# create a new file of all comments combined
df_concat.to_csv('/content/drive/MyDrive/USYD/SCDL1991/SCDL 1991 Group 10 Folder/Comments Data/TikTok.csv', index=False)

## Reddit

In [ ]:
# Reddit
def read_csv_with_custom_header(file):
    df_tmp = pd.read_csv(file_path + file, encoding="ISO-8859-1")
    # Extract URL from the second row and create a new column
    url = df_tmp.iloc[0, 1]

    df = pd.read_csv(file_path + file, header=6, encoding="ISO-8859-1")
    df['post_url'] = url

    # remove duplicates and empty
    df.drop_duplicates(subset=['Message'], inplace=True)
    df = df[df['Message'].notna()]

    # [deleted], removed
    df = df[df['Message'] != '[deleted]']
    df = df[df['Message'] != '[removed]']

    # Filter out non-English comments
    df = df[df['Message'].apply(is_english)]

    # sort by most recent to latest
    df['Date'] = pd.to_datetime(df['Date'], format='mixed')
    df.sort_values(by='Date', ascending=False, inplace=True)
    df = df.head(100)

    return df

df_concat = pd.concat([read_csv_with_custom_header(f) for f in file_list if os.path.isfile(file_path + f)], ignore_index=True)

field_dropped = ["unnamed", "Votes", "isPremium", "(view source)"]
for n in field_dropped:
  df_concat.drop(df_concat.columns[df_concat.columns.str.contains(n,case = False)],axis = 1, inplace = True)

df_concat.rename(columns={'Score': 'comment_likes'}, inplace=True)
df_concat.rename(columns={'Author': 'comment_author'}, inplace=True)
df_concat.rename(columns={'Message': 'comment_text'}, inplace=True)
df_concat.rename(columns={'Date': 'comment_time'}, inplace=True)

df_concat

<ipython-input-21-c24e182f71e7>:32: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_concat.drop(df_concat.columns[df_concat.columns.str.contains(n,case = False)],axis = 1, inplace = True)


,comment_author,comment_time,comment_likes,comment_text,post_url
0,isparavanje,2024-09-04,1.0,It doesn't work like this in the sense that JW...,https://www.reddit.com/r/AskPhysics/comments/1...
1,colossuscollosal,2024-09-04,1.0,has there been measures on what new mass is fo...,https://www.reddit.com/r/AskPhysics/comments/1...
2,isparavanje,2024-09-04,1.0,"No, it doesn't do anything regarding the evide...",https://www.reddit.com/r/AskPhysics/comments/1...
3,colossuscollosal,2024-09-04,1.0,thanks for the quick response - beyond tired l...,https://www.reddit.com/r/AskPhysics/comments/1...
4,isparavanje,2024-09-04,1.0,The tired light hypothesis that you're referri...,https://www.reddit.com/r/AskPhysics/comments/1...
...,...,...,...,...,...
1447,Fourier864,2015-06-08,1.0,Dark energy has a density of about 1 Joule per...,https://www.reddit.com/r/videos/comments/3fzyu...
1448,MaritMonkey,2015-06-08,1.0,"OK, it's not actually ""pain."" \n\nBut I don't...",https://www.reddit.com/r/videos/comments/3fzyu...
1449,Saerain,2015-06-08,1.0,I suppose you mean to imply that it should be ...,https://www.reddit.com/r/videos/comments/3fzyu...
1450,Saerain,2015-06-08,1.0,It must really suck to have pain receptors in ...,https://www.reddit.com/r/videos/comments/3fzyu...


In [ ]:
new_df = df_concat[['post_url']]
new_df.drop_duplicates(subset=['post_url'], inplace=True)

def extract_title_from_url(url):
  return url.split('/')[-2].replace("_", " ")

new_df['post_title'] = new_df['post_url'].apply(extract_title_from_url)
new_df

In [ ]:
new_df.to_csv('/content/drive/MyDrive/USYD/SCDL1991/SCDL 1991 Group 10 Folder/Comments Data/dict_reddit.csv', index=False)

In [ ]:
# create a new file of all comments combined
df_concat.to_csv('/content/drive/MyDrive/USYD/SCDL1991/SCDL 1991 Group 10 Folder/Comments Data/Reddit_new.csv', index=False)